In [16]:
import iatikit

In [3]:
iatikit.__version__

'2.1.0'

In [1]:
import sys

!{sys.executable} -m pip install --upgrade iatikit

  Found existing installation: iatikit 2.0.5
    Uninstalling iatikit-2.0.5:
      Successfully uninstalled iatikit-2.0.5


In [17]:
iatikit.download.data()

In [1]:
from datetime import datetime
import iatikit

registry = iatikit.data()

publishers = sorted(
    [(min([d.metadata.get('metadata_created')
           for d in p.datasets]
          ), p.metadata.get('title'))
     for p in registry.publishers])

for idx, tup in enumerate(publishers):
    print('{order}: {name} ({date})'.format(
        order=(idx + 1),
        name=tup[1],
        date=datetime.strptime(tup[0], '%Y-%m-%dT%H:%M:%S.%f').date()
    ))

1: UK - Department for International Development (DFID) (2011-01-29)
2: The William and Flora Hewlett Foundation (2011-03-31)
3: The World Bank (2011-05-14)
4: Development Initiatives Poverty Research (2011-07-13)
5: Australia - Department of  Foreign Affairs and Trade (2011-09-02)
6: Netherlands - Ministry of Foreign Affairs (2011-09-19)
7: United Nations Office for Project Services (UNOPS) (2011-10-03)
8: Sweden, through Swedish International Development Cooperation Agency (Sida) (2011-11-11)
9: Oxfam GB (2011-11-25)
10: Finland - Ministry of Foreign Affairs (2011-11-25)
11: The Global Fund to Fight AIDS, Tuberculosis and Malaria (2011-11-25)
12: Asian Development Bank (2011-11-28)
13: Publish What You Fund (2011-11-29)
14: Resource Extraction Monitoring (2012-02-21)
15: ADRA-UK (2012-02-24)
16: Catholic Agency For Overseas Development (CAFOD) (2012-03-21)
17: GAVI Alliance (2012-03-30)
18: GlobalGiving.org (2012-05-09)
19: LEAD International (2012-05-24)
20: MapAction (2012-05-28)
2

507: Trianglen (2017-04-28)
508: HealthNet TPO (2017-05-02)
509: Food and Agriculture Organization of the United Nations (FAO) (2017-05-03)
510: International Union for Conservation of Nature and Natural Resources (2017-05-15)
511: Meningitis Research Foundation (2017-05-18)
512: International Network for Bamboo and Rattan (INBAR) (2017-05-19)
513: Arab foundation for freedoms and equality (2017-05-24)
514: Near East Foundation (NEF USA) (2017-06-07)
515: World Economic Forum Geneva (2017-06-08)
516: World Health Organization (2017-06-08)
517: Giro555 (Stichting Samenwerkende Hulporganisaties) (2017-06-12)
518: Support Trust for Africa Development (STAD) (2017-06-13)
519: Netherlands Enterprise Agency (2017-06-15)
520: EMMS International (2017-06-23)
521: MVO Nederland (2017-06-27)
522: PUM Netherlands (2017-06-27)
523: People in Need (2017-06-28)
524: Disability Rights Advocacy Fund (2017-06-29)
525: AICS - Agenzia Italiana per la Cooperazione allo Sviluppo / Italian Agency for Cooper

In [37]:
import iatikit

# get the AsDB publisher
asdb = registry.publishers.find(name='asdb')

activities = asdb.activities

total_c = 0
total_d_e = 0

for activity in activities:  
    values = activity.etree.xpath('transaction[transaction-type/@code="C"]/value/text()')
    total_c += sum([float(v) for v in values])

for activity in activities:  
    values = activity.etree.xpath('transaction[transaction-type/@code="D" or transaction-type/@code="E"]/value/text()')
    total_d_e += sum([float(v) for v in values])

print(total_c)        
print(total_d_e)

156554932323.78998
92602825355.02998


In [14]:
import iatikit

registry = iatikit.data()

dfid = registry.publishers.find(name='dfid')
sector_category = iatikit.Sector(311, 2)  # Agriculture

ag_acts = dfid.activities.where(
    actual_start__lte='2017-12-31',  # started before 2018
    actual_end__gte='2017-01-01',  # ended after 2016
    sector__in=sector_category,
)
print('DFID had {:,} agricultural activities running during 2017.'.format(
    len(ag_acts)))

DFID had 201 agricultural activities running during 2017.


In [4]:
transaction_types = ['D', 'E', '3', '4']
' or '.join(['transaction-type/@code="{transaction_type}"'.format(
        transaction_type=transaction_type,
    ) for transaction_type in transaction_types])

'transaction-type/@code="D" or transaction-type/@code="E" or transaction-type/@code="3" or transaction-type/@code="4"'

In [9]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

def total_transactions(transaction_name, org_name, year):
    if transaction_name == 'commitments':
        transaction_types = ['C', '2']
    elif transaction_name == 'spend':
        transaction_types = ['D', 'E', '3', '4']
    else:
        raise Exception(f'Unknown transaction name: {transaction_name}')
    org = registry.publishers.find(name=org_name)
    activities = org.activities

    total = defaultdict(float)

    type_str = ' or '.join([f'transaction-type/@code="{transaction_type}"'
                            for transaction_type in transaction_types])
    
    path = f'transaction[{type_str}]' + \
           f'[number(translate(transaction-date/@iso-date, "-", "")) >= {year}0101 ' + \
           f'and number(translate(transaction-date/@iso-date, "-", "")) < {year+1}0101]' + \
           f'/value'
    for activity in activities:
        values = activity.etree.xpath(path)
        default_currency = activity.etree.get('default-currency')
        for value in values:
            currency = value.get('currency') or default_currency
            total[currency] += float(value.text)

    print(f'Sum of {transaction_name} for {org_name} in {year}:')
    if total == {}:
        print('  No transactions of that type were found')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': None,
            'value': None,
        }]
    else:
        for currency, value in total.items():
            print(f'  {currency} {value:,.0f}')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': currency,
            'value': value,
        } for currency, value in total.items()]
    
    return out

In [10]:
import csv
fieldnames = ['org_name', 'year', 'transaction_type', 'currency', 'value']

org_names = [
    'asdb',
    'undp',
    'dfid',
    'afdb',
    'millenniumchallenge',
    'worldbank',
    'iadb',
    'gac-amc',
    'unicef',
    'ec-echo',
    'ec-near',
    'gavi',
    'theglobalfund',
    'sida',
    'minbuza_nl',
    'usaid',
    'ec-devco',
    'be-dgd',
    'unitedstates',
    'bmz',
    'ausgov',
    'ebrd',
    'eib',
    'finland_mfa',
    'ifcwbg',
    'sdc_ch',
    'danida',
    'bmgf',
    'france',
    'afd',
    'aics',
    'norad',
    'irishaid',
    'jica',
    'odakorea',
    'maec',
    'fco',
    'unocha',
    'mfat',
    'mofa-jpn',

]
with open('output.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for org_name in org_names:
        for transaction_name in ['commitments', 'spend']:
            for year in [2017]:
                    output = total_transactions(transaction_name, org_name, year)
                    for o in output:
                        writer.writerow(o)

Sum of commitments for asdb in 2017:
  USD 16,890,374,000
Sum of spend for asdb in 2017:
  USD 10,338,893,286
Sum of commitments for undp in 2017:
  No transactions of that type were found
Sum of spend for undp in 2017:
  USD 4,181,721,365
Sum of commitments for dfid in 2017:
  GBP 12,535,833,806
Sum of spend for dfid in 2017:
  GBP 8,800,304,807
Sum of commitments for afdb in 2017:
  XDR 5,877,483,644
Sum of spend for afdb in 2017:
  XDR 4,832,476,083
Sum of commitments for millenniumchallenge in 2017:
  USD 1,052,824,067
Sum of spend for millenniumchallenge in 2017:
  USD 719,539,661


Sum of commitments for worldbank in 2017:
  No transactions of that type were found


Sum of spend for worldbank in 2017:
  No transactions of that type were found
Sum of commitments for iadb in 2017:
  USD 10,154,653,279
Sum of spend for iadb in 2017:
  USD 8,784,092,441
Sum of commitments for gac-amc in 2017:
  CAD 2,963,636,803
Sum of spend for gac-amc in 2017:
  CAD 3,125,954,154
Sum of commitments for unicef in 2017:
  USD 5,042,334,671
Sum of spend for unicef in 2017:
  USD 5,438,421,271
Sum of commitments for ec-echo in 2017:
  EUR 2,405,149,113
Sum of spend for ec-echo in 2017:
  EUR 2,260,820,545
Sum of commitments for ec-near in 2017:
  EUR 3,846,294,231


Sum of spend for ec-near in 2017:
  EUR 3,248,804,728


Sum of commitments for gavi in 2017:
  USD 1,123,257,705
Sum of spend for gavi in 2017:
  USD 1,270,382,406
Sum of commitments for theglobalfund in 2017:
  USD 0
Sum of spend for theglobalfund in 2017:
  USD 0


...

Sum of spend for france in 2017:
  EUR 3,830,415
Sum of commitments for afd in 2017:
  No transactions of that type were found
Sum of spend for afd in 2017:
  EUR 1,261,072,185


Sum of commitments for aics in 2017:
  EUR 998,552,640
Sum of spend for aics in 2017:
  EUR 448,070,449
Sum of commitments for norad in 2017:
  NOK 34,891,365,967
Sum of spend for norad in 2017:
  NOK 34,132,387,252
Sum of commitments for irishaid in 2017:
  No transactions of that type were found


Sum of spend for irishaid in 2017:
  EUR 207,870,298


Sum of commitments for jica in 2017:
  No transactions of that type were found


Sum of spend for jica in 2017:
  No transactions of that type were found
Sum of commitments for odakorea in 2017:
  KRW 2,904,589,990
Sum of spend for odakorea in 2017:
  No transactions of that type were found
Sum of commitments for maec in 2017:
  No transactions of that type were found
Sum of spend for maec in 2017:
  No transactions of that type were found
Sum of commitments for fco in 2017:
  No transactions of that type were found
Sum of spend for fco in 2017:
  GBP 786,802,095
Sum of commitments for unocha in 2017:
  No transactions of that type were found
Sum of spend for unocha in 2017:
  No transactions of that type were found
Sum of commitments for mfat in 2017:
  No transactions of that type were found
Sum of spend for mfat in 2017:
  NZD 90,878,890
Sum of commitments for mofa-jpn in 2017:
  No transactions of that type were found
Sum of spend for mofa-jpn in 2017:
  No transactions of that type were found


In [8]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

def total_transactions(transaction_name, org_name, year):
    if transaction_name == 'commitments':
        transaction_types = ['C', '2']
    elif transaction_name == 'spend':
        transaction_types = ['D', 'E', '3', '4']
    else:
        raise Exception(f'Unknown transaction name: {transaction_name}')
    org = registry.publishers.find(name=org_name)
    activities = org.activities

    total = defaultdict(float)

    type_str = ' or '.join([f'transaction-type/@code="{transaction_type}"'
                            for transaction_type in transaction_types])
    
    path = f'transaction[{type_str}]' + \
           f'[number(translate(transaction-date/@iso-date, "-", "")) >= {year}0406 ' + \
           f'and number(translate(transaction-date/@iso-date, "-", "")) < {year+1}0406]' + \
           f'/value'
    for activity in activities:
        values = activity.etree.xpath(path)
        default_currency = activity.etree.get('default-currency')
        for value in values:
            currency = value.get('currency') or default_currency
            total[currency] += float(value.text)

    print(f'Sum of {transaction_name} for {org_name} in {year}:')
    if total == {}:
        print('  No transactions of that type were found')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': None,
            'value': None,
        }]
    else:
        for currency, value in total.items():
            print(f'  {currency} {value:,.0f}')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': currency,
            'value': value,
        } for currency, value in total.items()]
    
    return out

In [9]:
import csv
fieldnames = ['org_name', 'year', 'transaction_type', 'currency', 'value']

org_names = [
    'beis',
    'fco',
    'defra_transparency',
    'dwp',
    'dhsc',
    'homeoffice',
    'ukmod',
    'dcms',
    'dfid',
    'dfe'
]
with open('output.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for org_name in org_names:
        for transaction_name in ['commitments', 'spend']:
            for year in [2019, 2018, 2017]:
                    output = total_transactions(transaction_name, org_name, year)
                    for o in output:
                        writer.writerow(o)

Sum of commitments for beis in 2019:
  No transactions of that type were found
Sum of commitments for beis in 2018:
  GBP 322,100,000
Sum of commitments for beis in 2017:
  GBP 494,983,801
Sum of spend for beis in 2019:
  GBP 80,834,005
  USD 4,000,000
Sum of spend for beis in 2018:
  GBP 368,113,119
  USD 8,163,335
Sum of spend for beis in 2017:
  GBP 412,480,683
  USD 9,875,000
Sum of commitments for fco in 2019:
  No transactions of that type were found
Sum of commitments for fco in 2018:
  No transactions of that type were found
Sum of commitments for fco in 2017:
  No transactions of that type were found
Sum of spend for fco in 2019:
  No transactions of that type were found
Sum of spend for fco in 2018:
  GBP 1,042,825,223
Sum of spend for fco in 2017:
  GBP 886,569,095
Sum of commitments for defra_transparency in 2019:
  GBP 12,257,860
Sum of commitments for defra_transparency in 2018:
  GBP 12,742,860
Sum of commitments for defra_transparency in 2017:
  GBP 42,000,000
Sum of sp

In [32]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

import csv
fieldnames = ['name', 'period-start', 'period-end', 'value', 'currency']

pub_names = [
    'asdb',
    'undp',
    'dfid',
    'afdb',
    'millenniumchallenge',
    'worldbank',
    'iadb',
    'gac-amc',
    'unicef',
    'ec-echo',
    'ec-near',
    'gavi',
    'theglobalfund',
    'sida',
    'minbuza_nl',
    'usaid',
    'ec-devco',
    'be-dgd',
    'unitedstates',
    'bmz',
    'ausgov',
    'ebrd',
    'eib',
    'finland_mfa',
    'ifcwbg',
    'sdc_ch',
    'danida',
    'bmgf',
    'france',
    'afd',
    'aics',
    'norad',
    'irishaid',
    'jica',
    'odakorea',
    'maec',
    'fco',
    'unocha',
    'mfat',
    'mofa-jpn',

]

rows = []

for pub_name in pub_names:
    pub = registry.publishers.get(pub_name)
# print(pub)
    orgs = list(pub.organisations)
    for org in orgs:
        default_currency = org.etree.xpath('@default-currency')
        default_currency = default_currency[0] if default_currency != [] else 'No default_currency provided'
        name = org.etree.xpath('name/narrative/text()')
        name = name[0] if name != [] else 'No name provided'
        totals = org.etree.xpath('total-expenditure')
        for total in totals:
            period_start = total.xpath('period-start/@iso-date')[0]
            period_end = total.xpath('period-end/@iso-date')[0]
            value = total.xpath('value/text()')[0]
            currency = total.xpath('value/@currency')
            currency = currency[0] if currency != [] else default_currency
            rows.append({
                'name': name, 
                'period-start': period_start, 
                'period-end': period_end, 
                'value': value,
                'currency': currency,
            })
            
# print(rows)
            
with open('totalspend.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

In [10]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

def total_transactions(transaction_name, org_name, year):
    if transaction_name == 'commitments':
        transaction_types = ['C', '2']
    elif transaction_name == 'spend':
        transaction_types = ['D', 'E', '3', '4']
    else:
        raise Exception(f'Unknown transaction name: {transaction_name}')
    org = registry.publishers.find(name=org_name)
    activities = org.activities

    total = defaultdict(float)

    type_str = ' or '.join([f'transaction-type/@code="{transaction_type}"'
                            for transaction_type in transaction_types])
    
    path = f'transaction[{type_str}]' + \
           f'[number(translate(transaction-date/@iso-date, "-", "")) >= {year}0101 ' + \
           f'and number(translate(transaction-date/@iso-date, "-", "")) < {year+1}0101]' + \
           f'/value'
    for activity in activities:
        values = activity.etree.xpath(path)
        default_currency = activity.etree.get('default-currency')
        for value in values:
            currency = value.get('currency') or default_currency
            total[currency] += float(value.text)

    print(f'Sum of {transaction_name} for {org_name} in {year}:')
    if total == {}:
        print('  No transactions of that type were found')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': None,
            'value': None,
        }]
    else:
        for currency, value in total.items():
            print(f'  {currency} {value:,.0f}')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': currency,
            'value': value,
        } for currency, value in total.items()]
    
    return out

In [11]:
import csv
fieldnames = ['org_name', 'year', 'transaction_type', 'currency', 'value']

org_names = [
    'unocha',
    'ocha_fts'
]
with open('output.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for org_name in org_names:
        for transaction_name in ['commitments', 'spend']:
            for year in [2018, 2017]:
                    output = total_transactions(transaction_name, org_name, year)
                    for o in output:
                        writer.writerow(o)

Sum of commitments for unocha in 2018:
  No transactions of that type were found
Sum of commitments for unocha in 2017:
  No transactions of that type were found
Sum of spend for unocha in 2018:
  No transactions of that type were found
Sum of spend for unocha in 2017:
  No transactions of that type were found
Sum of commitments for ocha_fts in 2018:
  No transactions of that type were found
Sum of commitments for ocha_fts in 2017:
  No transactions of that type were found
Sum of spend for ocha_fts in 2018:
  No transactions of that type were found
Sum of spend for ocha_fts in 2017:
  No transactions of that type were found


In [5]:
import iatikit

registry = iatikit.data()

beis = registry.publishers.find(name='beis')
print('BEIS has {:,} datasets.'.format(len(beis.datasets)))

BEIS has 28 datasets.


In [21]:
import iatikit

registry = iatikit.data()

beis = registry.publishers.find(name='beis')
print('BEIS has {:,} activities.'.format(len(beis.activities)))

BEIS has 2,348 activities.


In [24]:
import iatikit

registry = iatikit.data()

beis = registry.publishers.find(name='beis')
fco = registry.publishers.find(name='fco')
defra = registry.publishers.find(name='defra_transparency')
dwp = registry.publishers.find(name='dwp')
dhsc = registry.publishers.find(name='dhsc')
homeoffice = registry.publishers.find(name='homeoffice')
mod = registry.publishers.find(name='ukmod')
dcms = registry.publishers.find(name='dcms')
dfid = registry.publishers.find(name='dfid')
dfe = registry.publishers.find(name='dfe')

print('beis has {:,} activities.'.format(len(beis.activities)))
print('fco has {:,} activities.'.format(len(fco.activities)))
print('defra has {:,} activities.'.format(len(defra.activities)))
print('dwp has {:,} activities.'.format(len(dwp.activities)))
print('dhsc has {:,} activities.'.format(len(dhsc.activities)))
print('homeoffice has {:,} activities.'.format(len(homeoffice.activities)))
print('mod has {:,} activities.'.format(len(mod.activities)))
print('dcms has {:,} activities.'.format(len(dcms.activities)))
print('dfid has {:,} activities.'.format(len(dfid.activities)))
print('dfe has {:,} activities.'.format(len(dfe.activities)))

beis has 2,348 activities.
fco has 3,166 activities.
defra has 119 activities.
dwp has 1 activities.
dhsc has 116 activities.
homeoffice has 6 activities.
mod has 40 activities.
dcms has 2 activities.
dfid has 19,133 activities.
dfe has 2 activities.


In [35]:
import csv
fieldnames = ['org_name', 'year', 'transaction_type', 'currency', 'value']

org_names = [
    'beis',
    'fco',
    'defra_transparency',
    'dwp',
    'dhsc',
    'homeoffice',
    'ukmod',
    'dcms',
    'dfid',
    'dfe'
]
with open('output.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for org_name in org_names:
        output = activities()
        for o in output:
            writer.writerow(o)

TypeError: 'ActivitySet' object is not callable

In [31]:
import iatikit
from collections import defaultdict

registry = iatikit.data()
activities = org.activities

type_str = ' or '.join([f'activity/@code="{activity}"'
                            for activity in activities])

path = f'activity[{type_str}]' + \
           f'[number(translate(generated-datetime/@iso-date, "-", "")) >= 20190701 ' + \
           f'and number(translate(generated-datetime/@iso-date, "-", "")) < 20190801]' + \

beis = registry.publishers.find(name='beis')
total_july = len(beis.activities)
print('{:,} beis activities published in July 2019'.format(len(beis.activities)))

SyntaxError: invalid syntax (<ipython-input-31-b7c029d277e3>, line 13)